# Partie 3 – Entraînement du modèle (corrigé)
Ce notebook charge `datas/dataset.csv` (TSV), entraîne plusieurs régressions (linéaire, arbre de décision, forêt aléatoire), évalue en R², puis sauvegarde le **meilleur** modèle dans `modele/modele_pokemon.mod`.

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 140)
print("Pandas:", pd.__version__)

## Chargement du dataset (TSV)

In [ ]:
dataset_path = "datas/dataset.csv"
dataset = pd.read_csv(dataset_path, delimiter='\t')
print("Shape dataset:", dataset.shape)
display(dataset.head(3))

## Sélection des variables X (prédictives) et y (cible)

In [ ]:
feature_cols = [
    'POINTS_DE_VIE',
    'NIVEAU_ATTAQUE',
    'NIVEAU_DEFFENCE',           
    'NIVEAU_ATTAQUE_SPECIALE',
    'NIVEAU_DEFENSE_SPECIALE',
    'VITESSE',
    'GENERATION'
]


target_col = 'POURCENTAGE_DE_VICTOIRES'


missing_feat = [c for c in feature_cols if c not in dataset.columns]
if missing_feat:
    print('Colonnes manquantes dans le dataset:', missing_feat)
    raise KeyError('Colonnes features absentes : ' + ', '.join(missing_feat))
if target_col not in dataset.columns:
    raise KeyError(f"Colonne cible absente: {target_col}")


X = dataset[feature_cols].values
y = dataset[target_col].values

print('X shape:', X.shape, ' | y shape:', y.shape)

## Découpage apprentissage / validation

In [ ]:
X_APPRENTISSAGE, X_VALIDATION, Y_APPRENTISSAGE, Y_VALIDATION = train_test_split(
    X, y, test_size=0.2, random_state=0
)
print('Train:', X_APPRENTISSAGE.shape, 'Val:', X_VALIDATION.shape)

## Entraînement et évaluation des modèles (R²)

In [ ]:
results = []


lin = LinearRegression()
lin.fit(X_APPRENTISSAGE, Y_APPRENTISSAGE)
pred_lin = lin.predict(X_VALIDATION)
r2_lin = r2_score(Y_VALIDATION, pred_lin)
print(">> ----------- REGRESSION LINEAIRE -----------")
print(">> Precision (R²) =", r2_lin)
print("-----------------------------------------------")
results.append(('LinearRegression', lin, r2_lin))


tree = DecisionTreeRegressor(random_state=0)
tree.fit(X_APPRENTISSAGE, Y_APPRENTISSAGE)
pred_tree = tree.predict(X_VALIDATION)
r2_tree = r2_score(Y_VALIDATION, pred_tree)
print(">> ----------- ARBRE DE DECISION -------------")
print(">> Precision (R²) =", r2_tree)
print("-----------------------------------------------")
results.append(('DecisionTreeRegressor', tree, r2_tree))


rf = RandomForestRegressor(random_state=0)
rf.fit(X_APPRENTISSAGE, Y_APPRENTISSAGE)
pred_rf = rf.predict(X_VALIDATION)
r2_rf = r2_score(Y_VALIDATION, pred_rf)
print(">> ----------- FORET ALEATOIRE ---------------")
print(">> Precision (R²) =", r2_rf)
print("-----------------------------------------------")
results.append(('RandomForestRegressor', rf, r2_rf))


best_name, best_model, best_r2 = max(results, key=lambda x: x[2])
print(f">> Meilleur modèle: {best_name} avec R² = {best_r2:.4f}")

## Sauvegarde du meilleur modèle

In [ ]:
os.makedirs('modele', exist_ok=True)
model_path = 'modele/modele_pokemon.mod'
joblib.dump(best_model, model_path)
print('Modèle sauvegardé ->', model_path)